<a href="https://colab.research.google.com/github/atick-faisal/Crowd-Emotion/blob/main/src/CE_Cross_Check.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image_dataset_from_directory

In [2]:
test_dir = '/content/drive/MyDrive/Research/Crowd Emotion v3/Spectrograms/Log/'
train_dir = '/content/drive/MyDrive/Research/Crowd Emotion Val/Spectrograms/Log/'

In [3]:
BATCH_SIZE = 32
IMG_SIZE = (160, 160)

In [4]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    directory                 = train_dir,
    image_size                = IMG_SIZE,
    batch_size                = BATCH_SIZE
)

Found 9515 files belonging to 3 classes.


In [5]:
test_ds = tf.keras.preprocessing.image_dataset_from_directory(
    directory                 = test_dir,
    image_size                = IMG_SIZE,
    batch_size                = BATCH_SIZE
)

Found 6108 files belonging to 3 classes.


In [6]:
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.prefetch(buffer_size=AUTOTUNE)

In [7]:
preprocess_input = tf.keras.applications.mobilenet_v2.preprocess_input

In [8]:
rescale = tf.keras.layers.experimental.preprocessing.Rescaling(1./127.5, offset= -1)

In [9]:
IMG_SHAPE = IMG_SIZE + (3,)
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               weights='imagenet')

In [10]:
image_batch, label_batch = next(iter(train_ds))
feature_batch = base_model(image_batch)
print(feature_batch.shape)

(32, 5, 5, 1280)


In [11]:
base_model.trainable = False

In [12]:
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
feature_batch_average = global_average_layer(feature_batch)
print(feature_batch_average.shape)

(32, 1280)


In [13]:
prediction_layer = tf.keras.layers.Dense(3)
prediction_batch = prediction_layer(feature_batch_average)
print(prediction_batch.shape)

(32, 3)


In [14]:
inputs = tf.keras.Input(shape=(160, 160, 3))
x = preprocess_input(inputs)
x = base_model(x, training=False)
x = global_average_layer(x)
x = tf.keras.layers.Dropout(0.2)(x)
outputs = prediction_layer(x)
model = tf.keras.Model(inputs, outputs)

In [15]:
base_learning_rate = 0.001
model.compile(optimizer=tf.keras.optimizers.Adam(lr=base_learning_rate),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [16]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 160, 160, 3)]     0         
_________________________________________________________________
tf.math.truediv (TFOpLambda) (None, 160, 160, 3)       0         
_________________________________________________________________
tf.math.subtract (TFOpLambda (None, 160, 160, 3)       0         
_________________________________________________________________
mobilenetv2_1.00_160 (Functi (None, 5, 5, 1280)        2257984   
_________________________________________________________________
global_average_pooling2d (Gl (None, 1280)              0         
_________________________________________________________________
dropout (Dropout)            (None, 1280)              0         
_________________________________________________________________
dense (Dense)                (None, 3)                 3843  

In [17]:
history = model.fit(train_ds, epochs=10)

Epoch 1/10
298/298 [==============================] - 84s 265ms/step - loss: 0.3497 - accuracy: 0.8782
Epoch 2/10
298/298 [==============================] - 81s 265ms/step - loss: 0.0521 - accuracy: 0.9858
Epoch 3/10
298/298 [==============================] - 81s 263ms/step - loss: 0.0333 - accuracy: 0.9924
Epoch 4/10
298/298 [==============================] - 81s 264ms/step - loss: 0.0232 - accuracy: 0.9948
Epoch 5/10
298/298 [==============================] - 81s 265ms/step - loss: 0.0202 - accuracy: 0.9942
Epoch 6/10
298/298 [==============================] - 81s 264ms/step - loss: 0.0143 - accuracy: 0.9965
Epoch 7/10
298/298 [==============================] - 80s 263ms/step - loss: 0.0122 - accuracy: 0.9973
Epoch 8/10
298/298 [==============================] - 81s 265ms/step - loss: 0.0125 - accuracy: 0.9969
Epoch 9/10
298/298 [==============================] - 81s 265ms/step - loss: 0.0111 - accuracy: 0.9973
Epoch 10/10
298/298 [==============================] - 81s 264ms/step - l

In [18]:
loss, accuracy = model.evaluate(test_ds)

print("Loss: ", loss)
print("Accuracy: ", accuracy)

191/191 [==============================] - 21s 104ms/step - loss: 4.6184 - accuracy: 0.2341
Loss:  4.618438720703125
Accuracy:  0.23411919176578522


In [19]:
Loss:  5.110820293426514
Accuracy:  0.22059905529022217

Loss:  4.618438720703125
Accuracy:  0.23411919176578522